In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import json

In [3]:
# Define the data folder path
folder_path = './data/question_1a'
file_type = ".json"

with open(f'{folder_path}/appliance_params{file_type}', 'r') as f:
    appliance_params = json.load(f)

with open(f'{folder_path}/bus_params{file_type}', 'r') as f:
    bus_params = json.load(f)

with open(f'{folder_path}/consumer_params{file_type}', 'r') as f:
    consumer_params = json.load(f)

with open(f'{folder_path}/DER_production{file_type}', 'r') as f:
    DER_production = json.load(f)

with open(f'{folder_path}/usage_preference{file_type}', 'r') as f:
    usage_preference = json.load(f)


In [4]:
#System Parameters
bus_data = bus_params[0]
tau_imp = bus_data['import_tariff_DKK/kWh']
tau_exp = bus_data['export_tariff_DKK/kWh']
max_import = bus_data['max_import_kW']
max_export = bus_data['max_export_kW']
electricity_prices = bus_data['energy_price_DKK_per_kWh']

In [5]:
#PV parameters
pv_data = appliance_params['DER'][0]
pv_max_power = pv_data['max_power_kW']
pv_profile = DER_production[0]['hourly_profile_ratio']
pv_prod_hourly = [pv_max_power * ratio for ratio in pv_profile] # PV production per hour (kW) (No curtailment)

In [6]:
#Load parameters
load_data = appliance_params['load'][0]
E_min = usage_preference[0]['load_preferences'][0]['min_total_energy_per_day_hour_equivalent'] # Minimum daily consumption (kWh)

In [7]:
#Temporal parameters
T = len(electricity_prices)  # 24 hours
Times = range(T) # Time horizon (24 hours)

In [8]:
model = gp.Model("Energy_Optimization")

# Decision variables:
L_t = model.addVars(Times, lb=0, name="L_t")  # Load consumption (kW)
C_t = model.addVars(Times, lb=0, name="C_t")  # Energy curtailed from PV (kW)
G_imp_t = model.addVars(Times, lb=0, ub=max_import, name="G_imp_t")  # Grid import (kW)
G_exp_t = model.addVars(Times, lb=0, ub=max_export, name="G_exp_t")  # Grid export (kW)


Set parameter Username
Set parameter LicenseID to value 2617496
Academic license - for non-commercial use only - expires 2026-02-03


In [9]:
#Objective function
model.setObjective(
    gp.quicksum(electricity_prices[t] * (G_imp_t[t] - G_exp_t[t]) + 
                tau_imp * G_imp_t[t] + tau_exp * G_exp_t[t] for t in Times),
    GRB.MINIMIZE
)

In [10]:
#add constraints

Curtailment_constraint = [
	model.addLConstr(C_t[t] <= pv_prod_hourly[t], name=f"Curtailment")
	for t in Times
]

Load_coverage_constraint = [
	model.addLConstr(gp.quicksum(L_t[t] for t in Times) >= E_min, name=f"Total_Load_Coverage")
	for t in Times
]
# Power balance constraint
power_balance_constraint = [
	model.addLConstr(G_imp_t[t] - G_exp_t[t] == L_t[t] - pv_prod_hourly[t] + C_t[t], name="power_balance")
	for t in Times
]

In [11]:
#solve optimization problem
model.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 6800HS Creator Edition, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 72 rows, 96 columns and 696 nonzeros
Model fingerprint: 0x375edccd
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 8e+00]
  Bounds range     [5e+02, 1e+03]
  RHS range        [2e-01, 8e+00]
Presolve removed 72 rows and 96 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  0.000000000e+00


In [12]:
#print results
if model.status == GRB.OPTIMAL:
    print("\nOptimal solution found:")
    print(f"Total cost: {model.objVal:.2f} DKK")
    print("Hour | Load (kW) | Curtailment (kW) | Grid Import (kW) | Grid Export (kW) | PV Production (kW) | Electricity Price (DKK/kWh)")
    for t in Times:
        print(f"{t:4d} | {L_t[t].X:9.2f} | {C_t[t].X:15.2f} | {G_imp_t[t].X:15.2f} | {G_exp_t[t].X:15.2f} | {pv_prod_hourly[t]:18.2f} | {electricity_prices[t]:25.2f}")


Optimal solution found:
Total cost: 0.00 DKK
Hour | Load (kW) | Curtailment (kW) | Grid Import (kW) | Grid Export (kW) | PV Production (kW) | Electricity Price (DKK/kWh)
   0 |      0.00 |            0.00 |            0.00 |            0.00 |               0.00 |                      1.10
   1 |      0.00 |            0.00 |            0.00 |            0.00 |               0.00 |                      1.05
   2 |      0.00 |            0.00 |            0.00 |            0.00 |               0.00 |                      1.00
   3 |      0.00 |            0.00 |            0.00 |            0.00 |               0.00 |                      0.90
   4 |      0.00 |            0.00 |            0.00 |            0.00 |               0.00 |                      0.85
   5 |      0.15 |            0.00 |            0.00 |            0.00 |               0.15 |                      1.01
   6 |      0.42 |            0.00 |            0.00 |            0.00 |               0.42 |                

In [13]:
#sum L_t
print(f"\nTotal Load Consumption: {sum(L_t[t].X for t in Times):.2f} kWh")


Total Load Consumption: 13.47 kWh
